In [ ]:
import t3f
import tensorflow as tf
sess = tf.InteractiveSession()

## Tensor Nets

In this notebook we provide an example of how to build a simple Tensor Net (see https://arxiv.org/abs/1509.06569).

The main ingredient is the so-called TT-Matrix, a generalization of the Kronecker product matrices, i.e. matrices of the form 
$$A = A_1 \otimes A_2 \cdots \otimes A_n$$

In `t3f` TT-Matrices are represented using the `TensorTrain` class.

In [ ]:
W = t3f.random_matrix([[4, 7, 4, 7], [5, 5, 5, 5]], tt_rank=2)

print(W)

Using TT-Matrices we can compactly represent densely connected layers in neural networks, which allows us to greatly reduce number of parameters. Matrix multiplication can be handled by the `t3f.matmul` method which allows for multiplying dense (ordinary) matrices and TT-Matrices. Very simple neural network could look as following (for initialization several options such as `t3f.initializers.glorot`, `t3f.initializers.he` or `t3f.random_matrix` are available):

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int64, [None])

initializer = t3f.initializers.glorot([[4, 7, 4, 7], [5, 5, 5, 5]], tt_rank=2)
W1 = t3f.get_variable('W1', initializer=initializer) 
b1 = tf.get_variable('b1', shape=[625])
h1 = t3f.matmul(x, W1) + b1
h1 = tf.nn.relu(h1)

W2 = tf.get_variable('W2', shape=[625, 10])
b2 = tf.get_variable('b2', shape=[10])
h2 = tf.matmul(h1, W2) + b2

y_ = tf.one_hot(y, 10)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=h2))

For convenience we have implemented a layer analogous to *Keras* `Dense` layer but with a TT-Matrix instead of an ordinary matrix. An example of fully trainable net is provided below.

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from utils import TTDense

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Some preprocessing...

In [ ]:
x_train = x_train / 127.5 - 1.0
x_test = x_test / 127.5 - 1.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(TTDense(row_dims=[7, 4, 7, 4], column_dims=[5, 5, 5, 5], tt_rank=4, init='glorot', activation='relu', bias_init=1e-3))
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Note that in the dense layer we only have $1725$ parameters instead of $784 * 625 = 490000$.

In [ ]:
optimizer = optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))